In [1]:
import pandas as pd
import numpy as np
from datetime import timedelta
import random

# ---------- 1. CARGA Y LIMPIEZA INICIAL ----------
df = pd.read_csv("Accidentes de tránsito en carreteras-2020-2021-Sutran.csv", encoding="latin1", delimiter=";")

# Eliminar columnas irrelevantes
df = df.drop(columns=["FECHA_CORTE"])

# Convertir FECHA a tipo datetime
df["FECHA"] = pd.to_datetime(df["FECHA"], format="%Y%m%d", errors="coerce")
df["AÑO"] = df["FECHA"].dt.year
df["MES"] = df["FECHA"].dt.month
df["DIA"] = df["FECHA"].dt.day

# Convertir HORA a entero
df["HORA"] = pd.to_datetime(df["HORA"], format="%H:%M", errors="coerce").dt.hour

# Convertir a numérico
df["KILOMETRO"] = pd.to_numeric(df["KILOMETRO"], errors="coerce")
df["FALLECIDOS"] = pd.to_numeric(df["FALLECIDOS"], errors="coerce")
df["HERIDOS"] = pd.to_numeric(df["HERIDOS"], errors="coerce")

# Eliminar filas incompletas
df = df.dropna(subset=["FECHA", "HORA", "DEPARTAMENTO", "CODIGO_VÍA", "KILOMETRO", "MODALIDAD", "FALLECIDOS", "HERIDOS"])

# Reset index para evitar errores
df = df.reset_index(drop=True)

# ---------- 2. FUNCIÓN PARA GENERAR DATOS SINTÉTICOS ----------
def generar_sinteticos(df_original, cantidad_deseada):
    df_nuevo = []
    n_original = len(df_original)

    for _ in range(cantidad_deseada):
        fila = df_original.iloc[random.randint(0, n_original - 1)].copy()

        # Variar ligeramente los valores numéricos
        fila["KILOMETRO"] += np.random.normal(0, 2)
        fila["KILOMETRO"] = max(fila["KILOMETRO"], 0)

        # Variar la hora dentro del mismo día
        fila["HORA"] = min(max(fila["HORA"] + random.randint(-2, 2), 0), 23)

        # Modificar fecha aleatoriamente
        delta_dias = random.randint(-365, 365)
        fila["FECHA"] = fila["FECHA"] + timedelta(days=delta_dias)
        fila["AÑO"] = fila["FECHA"].year
        fila["MES"] = fila["FECHA"].month
        fila["DIA"] = fila["FECHA"].day

        # Leve aleatoriedad en víctimas
        fila["FALLECIDOS"] = max(0, int(fila["FALLECIDOS"] + np.random.choice([0, 1, -1], p=[0.8, 0.1, 0.1])))
        fila["HERIDOS"] = max(0, int(fila["HERIDOS"] + np.random.choice([0, 1, 2, -1], p=[0.7, 0.1, 0.1, 0.1])))

        df_nuevo.append(fila)

    return pd.DataFrame(df_nuevo)

# ---------- 3. GENERAR NUEVOS REGISTROS ----------
objetivo = 1_000_000
faltan = objetivo - len(df)

df_sintetico = generar_sinteticos(df, faltan)

# Unimos los datos reales + sintéticos
df_completo = pd.concat([df, df_sintetico], ignore_index=True)

# ---------- 4. CLASE DE ACCIDENTE (opcional para clasificación) ----------
df_completo["GRAVEDAD"] = df_completo["FALLECIDOS"] + df_completo["HERIDOS"]
df_completo["CLASE_ACCIDENTE"] = pd.cut(df_completo["GRAVEDAD"], bins=[-1, 0, 3, 100], labels=["Leve", "Moderado", "Grave"])

# ---------- 5. EXPORTAR ----------
df_completo.to_csv("accidentes_completo.csv", index=False)
print(f"Archivo generado con {len(df_completo):,} registros.")


Archivo generado con 1,000,000 registros.
